# NB #1:  Pool Exploration 

### Pool address: 0x8b6e6E7B5b3801FEd2CaFD4b22b8A16c2F2Db21a  -- 80% WETH - 20% DAI 

The Balancer Pool cadCAD model provides a core infrastructure for simulating Balancer Pools in the **Token Engineering Design and Verification process**.

It allows to simulate Balancer Pool transactions and verify new designs and application cases.  
In this notebook we plug **historical on-chain data** to a cadCAD model, and analyse **Pool Power** and **Pool Characteristics**.

For more information check out the [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/).

# A. System Context

### A1.1 System Specification
System specification details are available in the Balancer Simulations documentation:
- [Differential Specification](https://token-engineering-balancer.gitbook.io/balancer-simulations/balancer-simulations/v10nboverview)
- [Mathematical Specification](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/balancer-the-python-edition/balancer_math.py)
- [Model Architecture](https://token-engineering-balancer.gitbook.io/balancer-simulations/balancer-simulations/v10nboverview)
- [Naming Convention](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/naming-convention)

# B. cadCAD Simulations

### B1.1 Dependencies

In [ ]:
import pandas as pd 
from cadCAD.configuration.utils import config_sim

### B1.2 Initialize Pool


In this section you specify the Balancer Pool to be analyzed and run scripts to produce the'genesis_state', based on on-chain data. 

More information here: [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/onchaintransactions).  

In [1]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

from decimal import Decimal

# Spot price reference must be a symbol of a token in the pool in ALL_CAPS, you can ignore the spot price parameter for the simulations in this notebook.
parameters = {
    'spot_price_reference': ['DAI'],
}


#### Import Pool Transactions and Initialize Pool state

Genesis state, pool transactions, and (external) USD token prices are pulled from on-chain/API data collected in a .json file.  
**Choose *your* Balancer Pool, and produce this .json file first, then add the file path below.**

Learn more how to create this file using parsing scripts in the [documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/additional-code-and-instructions/onchaintransactions).


In [ ]:
from model.genesis_states import generate_initial_state

initial_values = generate_initial_state(initial_values_json='data/0x8b6e6e7b5b3801fed2cafd4b22b8a16c2f2db21a-initial_pool_states-prices.json', spot_price_base_currency=parameters['spot_price_reference'][0])

#### State Variables and Initial Values

In [ ]:
print('## State Variables')
print('# Pool')
pool = initial_values['pool']
pp.pprint(initial_values)


#### External USD Token Prices

In [ ]:
print('# External token prices, initial state')
token_prices = initial_values['token_prices']
pp.pprint(token_prices)

print('# Action Type')
action_type = initial_values['action_type']
pp.pprint(action_type)


### B1.3 State Update Functions & Policies

Balancer Simulations replicate Balancer Pool Transactions in state update functions.
For a detailed description on how we implement transactions, please visit the [Balancer Simulations documentation/Balancer Pool Functions](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/additional-code-and-instructions/balancer-the-python-edition/balancer-pool-functions).

### B1.4 Partial State Update Blocks

Partial State Update Blocks combine the following steps:  
1. **Initialize pool**, generate genesis state
2. Compute subsequent **actions in discrete timesteps**, store datetime and update the pool state variables
3. Update external **USD prices in discrete timesteps**, and store datetime  

For more information please visit the [Balancer Simulations documentation/V1.0 Model Overview](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/balancer-simulations/v10nboverview).

In [ ]:
from model.partial_state_update_block import generate_partial_state_update_blocks

result = generate_partial_state_update_blocks('data/0x8b6e6e7b5b3801fed2cafd4b22b8a16c2f2db21a-actions-prices.json')
partial_state_update_blocks = result['partial_state_update_blocks']
pp.pprint(partial_state_update_blocks)

### B1.5 Configuration

[cadCAD simulations](https://github.com/cadCAD-org/cadCAD/blob/master/documentation/README.md) support Monte Carlo runs, and parameter sweeps which are not applied in this notebook and can be ignored here.  
Timesteps 'T' are defined by your pool's .json file, no need to make edits here.

In [2]:
steps_number = result['steps_number']
print('# Steps ', steps_number)
sim_config = config_sim(
    {
        'N': 1,  # number of monte carlo runs
        'T': range(steps_number - 1),  # number of timesteps
        'M': parameters,  # simulation parameters
    }
)

NameError: name 'result' is not defined

### B1.6 Execution

In [ ]:
from model.sim_runner import *

df = run(initial_values, partial_state_update_blocks, sim_config)

### B1.7 Simulation Output Preparation

Post-processing (utils.py) splits up state variable dictionaries, and adds metrics to the data frame, such as 
- `token_total_value` (TVL)
- `generated_fees_(tokensymbol)` (fee collected per transaction)
- `token_k_values`  (token value in USD)  
based on  
- `token_k_balances`
- `token_k_price`  



In [ ]:
from model.parts.utils import post_processing

p_df = post_processing(df, include_spot_prices=False)

p_df.head()

# C. Simulation Outcome & Pool Exploration

Below we show a range of plots exploring pool states in the simulation:

**a) Pool Power:**  
- C1.1 TVL (Total Value Locked, over time) compared to  
- C1.1 Pool Size Growth (Number of tokens in the pool, over time)  
- C1.2 Token Balances (individual balances, over time)

**b) Pool Characteristics:**  
- C1.3 Source of Pool Growth (Total Growth vs. Fees collected)
- C1.4 Token Ratio
- C1.5 Action Types

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
print("Observation Time")
print(f"Start: {(p_df.iloc[0]['change_datetime'])}")
print(f"End: {(p_df.iloc[-1]['change_datetime'])}")
print("\n")
print(f"Total No. of Timesteps (incl. Price Updates): {(p_df.iloc[-1]['timestep'])}")
print(f"Total No. of Transactions: {len(p_df[p_df.action_type != 'external_price_update'])}")

### C1.1 TVL & Pool Size Growth

In [ ]:
#TVL vs. Pool Size Growth
tvl_p_df = p_df[['timestep', 'tvl', 'total_token_balances']] 
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(go.Scatter(x=tvl_p_df['timestep'],y=tvl_p_df['tvl'], line=dict(color='#7DFD64'), name="TVL total_token_value"), secondary_y=False,)
fig.add_trace(go.Scatter(x=tvl_p_df['timestep'],y=tvl_p_df['total_token_balances'], line=dict(color='#2C1839'), name="total_token_balance"), secondary_y=True,)
#Layout
fig.update_layout(title_text="<b>TVS vs. Pool Size Growth</b>")
fig.update_xaxes(title_text="timestep")
fig.update_yaxes(title_text="<b>TVL total_token_value</b> in USD", secondary_y=False)
fig.update_yaxes(title_text="<b>total_token_balance</b> in #", secondary_y=True)
fig.show()  

In [ ]:
#TVL growth over observation time in %
#Total token balance growth over observation time in %

**Observations:**
- both TVL and Pool Size grow, but
- Pool Size growth is huge - 
- USD value spikes around timestep 5000

### C1.2 Token Balances

In [ ]:
k = 2 #define number of tokens in your pool
fig = make_subplots(rows=k, cols=1)
fig.add_trace(go.Scatter(x=p_df['timestep'], y=p_df['token_weth_balance'], line=dict(color='#015B99'), name='token_weth_balance'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_df['timestep'], y=p_df['token_dai_balance'], line=dict(color='#5CB1EC'), name='token_dai_balance'), row=2, col=1)
fig.update_layout(height=400, width=1000, title_text="<b>Token Balances in #</b>")
fig.show()

**Observations:**
- ETH declines, DAI grows significantly
- Pool Size growth mainly driven by DAI growth
- Hypothesis: pool's main purpose is to serve as an exchange ETH for DAI (see also Action Types below)

### C1.3 Sources of Growth 

In [ ]:
#Pool Shares
ps_p_df = p_df[['timestep','pool_shares']]
fig = px.line(ps_p_df, x=ps_p_df['timestep'],y=ps_p_df['pool_shares'])
fig.update_layout(height=300, width=1000, title_text="<b>Pool Shares (BPT)</b>")
fig.update_xaxes(title_text="timestep")
fig.update_yaxes(title_text="<b>pool shares</b> in #", range=[99.985,101.00])
fig.show()

**Observations:**
- minor growth via liquidity deposits
- Hypothesis: fees are main source of Pool Growth

In [ ]:
#Fees
k = 2 #define number of tokens in your pool
fig = make_subplots(rows=k, cols=1)
fee_p_df = p_df[['timestep','generated_fees_dai','token_dai_price','generated_fees_weth','token_weth_price']]
fig.add_trace(go.Scatter(x=p_df['timestep'], y=p_df['generated_fees_weth'], line=dict(color='#015B99'), name='fees generated WETH'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_df['timestep'], y=p_df['generated_fees_dai'], line=dict(color='#5CB1EC'), name='fees generated DAI'), row=2, col=1)
fig.update_layout(height=400, width=1000, title_text="<b>Fees Generated over time</b>")

In [ ]:
p_df.info()

In [ ]:
#balance growth (total observation period)
daib_df = (p_df.iloc[-1][['token_dai_balance']])-(p_df.iloc[0][['token_dai_balance']]).copy()
daib_df = daib_df.reset_index()
daib_df.columns =['source', 'growth']
wethb_df = (p_df.iloc[-1][['token_weth_balance']])-(p_df.iloc[0][['token_weth_balance']]).copy()
wethb_df = wethb_df.reset_index()
wethb_df.columns =['source', 'growth']

#fee growth ACHTUNG add "generated_fees_k", is currently an object, this is why I use price atm
daif_df = p_df.groupby('action_type').sum()['token_dai_price'].drop(['exit', 'exit_swap', 'external_price_update','pool_creation']).reset_index()
daif_df.columns =['source', 'growth'] 
wethf_df = p_df.groupby('action_type').sum()['token_weth_price'].drop(['exit', 'exit_swap', 'external_price_update','pool_creation']).reset_index()
wethf_df.columns =['source', 'growth'] 

#merge
dai_df= pd.concat([daib_df, daif_df], ignore_index=True)
dai_df
weth_df= pd.concat([wethb_df, wethf_df], ignore_index=True)
weth_df

#plot chart
k = 2 #define number of tokens in your pool
fig = make_subplots(rows=1, cols=k, subplot_titles=("DAI Growth", "WETH Growth"))
fig.add_trace(go.Bar(name="DAI Growth", y=dai_df['growth'], x=dai_df['source'], marker_color=px.colors.sequential.Turbo), row=1, col=1)
fig.add_trace(go.Bar(name="WETH Growth", y=weth_df['growth'], x=weth_df['source'], marker_color=px.colors.sequential.Turbo), row=1, col=2)
fig.update_layout(height=400, width=1000, title_text="<b>Growth & Sources of Growth</b>", showlegend=False)

fig.show()

In [ ]:
# Compare USD value of fees collected ETH/DAI
# Compare % of USD value total fees collected via DAI vs. ETH

### C1.4 Token Ratio

In [ ]:
# TODO: create subplots
# TODO: alternatively consider line chart showing changes across particular events

events = [0, -1]
r_p_df = p_df.iloc[events][['token_dai_balance','token_weth_balance']].transpose(copy=True).reset_index()
r_p_df

fig1 = px.pie(r_p_df, values=0, names='index', color_discrete_sequence=px.colors.sequential.Turbo)
fig2 = px.pie(r_p_df, values=25601, names='index', color_discrete_sequence=px.colors.sequential.Turbo)
fig1.update_layout(height=300, width=1000, title_text="<b>Token Ratio</b>")
fig2.update_layout(height=300, width=1000, title_text="<b>Token Ratio</b>")

fig1.show()
fig2.show()

### C1.5 Action Types

In [ ]:
#action type per timestep
fig = px.scatter(p_df, x='timestep', y='total_token_balances', color='action_type', color_discrete_sequence=px.colors.sequential.Turbo, category_orders={'action_type': ['pool_creation', 'join', 'join_swap', 'swap', 'exit_swap', 'exit', 'external_price_update']})
fig.update_layout(height=400, width=1000, title_text="<b>Action Type / Timestep</b>")
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
# Show Action Types
a_p_df = p_df.groupby('action_type').size().to_frame('count').reset_index()
fig = px.pie(a_p_df, values='count', names='action_type', title='<b>Action Types</b>', hole=.3, color_discrete_sequence=px.colors.sequential.Turbo)
fig.show()
print(a_p_df)

**Observations:**

- Hypothesis confirmed, pool's main purpose is to serve as an exchange ETH for DAI (see Pool Size Growth)
